# Multi Layer Perceptron Model

- BOOK: [Predict the Future with MLPs, CNNs and LSTMs in Python](https://machinelearningmastery.com/deep-learning-for-time-series-forecasting/) by Jason Brownlee

In [1]:
# !pip3 uninstall -y torch torchvision
# !pip3 install torch torchvision

In [2]:
import os
import sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()

# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../framework_for_time_series_data/tslearn/'))
from collections import namedtuple
from data_loader import build_stock_uts
from ts_models import Model, MLP, EvaluationMetric
from time_series import TimeSeriesFactory, UnivariateTimeSeries

In [3]:
observations = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [4]:
def convert_uts_sequence_to_sml(uts_observations, prior_observations, forecasting_step):
    """Splits a given UTS into multiple input rows where each input row has a specified number of timestamps and the output is a single timestamp.
    
    Parameters:
    uts_observations -- 1D np array (of UTS data to transform to SML data with size  b rows/length x 1 dimension)
    prior_observations -- py int (of all observations before we get to where we want to start making the predictions)
    forecasting_step -- py int (of how far out to forecast, 1 only the next timestamp, 2 the next two timestamps, ... n the next n timestamps)
    
    Return:
    agg.values -- np array (of new sml data)
    """
    
    df = pd.DataFrame(uts_observations)
    cols = list()
    
    lag_col_names = []
    count_lag = 0
    # input sequence (t-n, ... t-1)
    for prior_observation in range(prior_observations, 0, -1):
        # print("prior_observation: ", prior_observation)
        cols.append(df.shift(prior_observation))
        new_col_name = "t-" + str(prior_observation)
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, forecasting_step):
        cols.append(df.shift(-i))
        new_col_name = "t"
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
        # put it all together
        uts_sml_df = pd.concat(cols, axis=1) 
        uts_sml_df.columns=[lag_col_names]
        # drop rows with NaN values
        uts_sml_df.dropna(inplace=True)
    
    # print(uts_sml_df)
    
    # colums to use to make prediction for last col
    X_train = uts_sml_df.iloc[:, 0: -1]
    
    # last column
    y_train = uts_sml_df.iloc[:, [-1]]
    return uts_sml_df, X_train, y_train

In [5]:
x_y, x, y = convert_uts_sequence_to_sml(observations, 3, 1)

In [6]:
x_y

,t-3,t-2,t-1,t
3,10.0,20.0,30.0,40
4,20.0,30.0,40.0,50
5,30.0,40.0,50.0,60
6,40.0,50.0,60.0,70
7,50.0,60.0,70.0,80
8,60.0,70.0,80.0,90


In [7]:
x

,t-3,t-2,t-1
3,10.0,20.0,30.0
4,20.0,30.0,40.0
5,30.0,40.0,50.0
6,40.0,50.0,60.0
7,50.0,60.0,70.0
8,60.0,70.0,80.0


In [8]:
y

,t
3,40
4,50
5,60
6,70
7,80
8,90


In [9]:
mlp_model = MLP()
X_y_df, X, y = mlp_model.augment_data(observations, 3, 1)

In [11]:
X_y_df

,t-3,t-2,t-1,t
3,10.0,20.0,30.0,40
4,20.0,30.0,40.0,50
5,30.0,40.0,50.0,60
6,40.0,50.0,60.0,70
7,50.0,60.0,70.0,80
8,60.0,70.0,80.0,90
